In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_recommenders as tfrs

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

2024-06-27 17:25:19.093531: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-27 17:25:19.630767: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 17:25:20.807447: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# RUTA 
DIR_DATA_LIMPIA = '../../datos/data-limpia'

# IMPORTAR DATA

EJERCICIOS = 'catalogo-de-ejercicios-onehotencoded.csv'
ejercicios = pd.read_csv(f"{DIR_DATA_LIMPIA}/{EJERCICIOS}", sep=",", encoding="latin1")

M1 = 'matriz-ejercicios-1.csv'
M2021 = 'matriz-ejercicios-2021.csv'
df_matrix_e1 = pd.read_csv(f"{DIR_DATA_LIMPIA}/{M1}", sep=",", encoding="latin1")
df_matrix_e2 = pd.read_csv(f"{DIR_DATA_LIMPIA}/{M2021}", sep=",", encoding="latin1")

In [3]:
def maxtrix_fact_to_ratings(df_exercises, df_matrix):
        rows = []
        for _, row_matrix in df_matrix.iterrows():
                user_id = row_matrix['rut']
                for exercise in df_matrix.columns[1:]:
                        if row_matrix[exercise] == 1:
                                row_exercise = df_exercises.iloc[int(exercise.lstrip('e'))]
                                new_row = {'rut': user_id}
                                new_row.update(row_exercise.to_dict())
                                rows.append(new_row)
        df = pd.DataFrame(rows)      
        return df

tomarlos todos -> todos los que aprobaron

In [5]:
ejercicios['score'] = ejercicios.iloc[:, 3:15].apply(lambda x: int(''.join(map(str, x)), 2), axis=1)

df_ejercicios = ejercicios.drop(labels=['idx', 'nombre', ], axis=1) # 'h4','h3','h2','h1','s4','s3','s2','s1','k4','k3','k2','k1'

df_ejercicios

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,score
0,2437df93d3f44a87b00834072aeb1ab0,0,0,0,1,0,0,0,0,0,0,0,1,257
1,31ea1c1b12174428b5a67a6576627de9,0,0,0,1,0,0,0,0,0,0,0,1,257
2,89f44e7f5842479fb283e43c52ce067b,0,0,0,1,0,0,0,0,0,0,0,1,257
3,718578451f3f4eca87437cadfe98d688,0,0,0,1,0,0,0,0,0,0,1,0,258
4,80c61dae74fa4915bf272ab17dfa62ff,0,0,0,1,0,0,0,0,0,0,1,0,258
5,171b5e86d4fb47268f2692587fbec073,0,0,0,1,0,0,0,0,0,0,1,1,259
6,46850a246d48484b8f104f8aab5679b6,0,0,0,1,0,0,0,0,0,0,1,1,259
7,729d37da8f2d46f3af2d891df04949ef,0,0,0,1,0,0,0,0,0,0,1,1,259
8,8f24397e36034cccb71e9d578975c33d,0,0,0,1,0,0,0,0,0,0,1,1,259
9,a3963220090f4e50a266ce53d33b9841,0,0,0,1,0,0,0,0,0,0,1,1,259


+ promedio de los estudiantes
+ recompensa -> promedio de notas -> realizaron el ejercicio


In [5]:
df1 = maxtrix_fact_to_ratings(df_ejercicios, df_matrix_e1)
df2 = maxtrix_fact_to_ratings(df_ejercicios, df_matrix_e2)

In [6]:
df1

,rut,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,score
0,0,2437df93d3f44a87b00834072aeb1ab0,0,0,0,1,0,0,0,0,0,0,0,1,257
1,0,31ea1c1b12174428b5a67a6576627de9,0,0,0,1,0,0,0,0,0,0,0,1,257
2,0,89f44e7f5842479fb283e43c52ce067b,0,0,0,1,0,0,0,0,0,0,0,1,257
3,0,718578451f3f4eca87437cadfe98d688,0,0,0,1,0,0,0,0,0,0,1,0,258
4,0,80c61dae74fa4915bf272ab17dfa62ff,0,0,0,1,0,0,0,0,0,0,1,0,258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6482,466,db7987d040dc469a9c247d54dd72939a,0,0,1,0,0,0,0,1,0,0,1,1,531
6483,466,4d5ed23727c04a8790f2e4ab3f09767e,0,0,1,0,0,0,1,0,0,1,1,1,551
6484,466,7da6cedd04c44a15b5e421440253acff,0,0,1,0,0,0,1,0,0,1,1,1,551
6485,466,9e02c76901da440ba1b70f0a75d8ae89,0,0,1,0,0,0,1,0,0,1,1,1,551


In [7]:
# Dividir en conjuntos de entrenamiento y prueba
train_data_1, test_data_1 = train_test_split(df1, test_size=0.2, random_state=42)

In [8]:
# Normalizar características numéricas
normalizar = MinMaxScaler()
train_data_1[['score']] = normalizar.fit_transform(train_data_1[['score']])
test_data_1[['score']] = normalizar.transform(test_data_1[['score']])

In [9]:
# Preparar datos de entrada y salida
X_train = train_data_1.drop(['rut', 'oid'], axis=1).values
Y_train = train_data_1['rut'].values

X_test = test_data_1.drop(['rut', 'oid'], axis=1).values
Y_test = test_data_1['rut'].values

In [10]:
# Definir el modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # salida de regresión
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Resumen del modelo
model.summary()

/home/edgar-ramxs/sistema-de-recomendaciones/.sistema-de-recomendaciones-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,009 (11.75 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Entrenar el modelo
history = model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
130/130 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 71413.7656 - val_loss: 58049.9531
Epoch 2/50
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 41713.9805 - val_loss: 19342.9453
Epoch 3/50
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19594.6055 - val_loss: 18051.0801
Epoch 4/50
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 18965.0781 - val_loss: 17739.3379
Epoch 5/50
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 18973.4629 - val_loss: 17615.2012
Epoch 6/50
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 18741.5039 - val_loss: 17571.5020
Epoch 7/50
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19030.3516 - val_loss: 17573.7676
Epoch 8/50
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 18227.6016 - val_loss: 17384.7148
Epoch 9/50
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 18151.2402 - val_loss: 17387.9551
Epoch 10/50
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 18543.8301 - val_loss: 17411.2754
Epoch 11/50
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss

In [12]:
# Evaluación del modelo
loss = model.evaluate(X_test, Y_test)
print(f'Loss en el conjunto de prueba: {loss}')

# Utilizar el modelo para hacer predicciones
predictions = model.predict(X_test)

41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17908.6582
Loss en el conjunto de prueba: 18352.970703125
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [13]:
# PREDICCIONES

# random user
user_id = random.randint(1, df1['rut'].unique().max())
print(f'Usuario: {user_id}\n')

items_interacted = df1[df1['rut'] == user_id]['oid'].values
items_interacted

Usuario: 103



array(['2437df93d3f44a87b00834072aeb1ab0',
       '89f44e7f5842479fb283e43c52ce067b',
       '718578451f3f4eca87437cadfe98d688',
       '80c61dae74fa4915bf272ab17dfa62ff',
       '171b5e86d4fb47268f2692587fbec073',
       'a3963220090f4e50a266ce53d33b9841',
       'ac7382763e484d37908da54c076f7577',
       'd8395f43e4a1454d90346ac5a1ba561a',
       '7f60644b0a1b484681ae5c8e36166c58',
       '52620b0c858a4c59bc324b65278d28bd',
       '0973dae0e1b74ab8baa8d94339ee3ae6',
       '16f619db31204ded9418136c4587ddd8',
       'db7987d040dc469a9c247d54dd72939a'], dtype=object)

In [14]:
# Obtener ítems no interactuados
items_not_interacted = df_ejercicios[~df_ejercicios['oid'].isin(items_interacted)]
items_not_interacted

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,score
1,31ea1c1b12174428b5a67a6576627de9,0,0,0,1,0,0,0,0,0,0,0,1,257
6,46850a246d48484b8f104f8aab5679b6,0,0,0,1,0,0,0,0,0,0,1,1,259
7,729d37da8f2d46f3af2d891df04949ef,0,0,0,1,0,0,0,0,0,0,1,1,259
8,8f24397e36034cccb71e9d578975c33d,0,0,0,1,0,0,0,0,0,0,1,1,259
14,baf2f8e0167a4e089d2cec16582c9ae9,0,0,0,1,0,0,0,1,0,0,1,1,275
17,29f15ef8dc32426f945f64e28c910a57,0,0,1,0,0,0,0,1,0,0,1,1,531
19,3d7d9e64ac0846caadc08dccf2538f55,0,0,1,0,0,0,1,0,0,1,1,1,551
20,4d51d13b9a2848a2803f7d1143fde6d4,0,0,1,0,0,0,1,0,0,1,1,1,551
21,4d5ed23727c04a8790f2e4ab3f09767e,0,0,1,0,0,0,1,0,0,1,1,1,551
22,7da6cedd04c44a15b5e421440253acff,0,0,1,0,0,0,1,0,0,1,1,1,551


In [15]:
# Normalizar características numéricas
normalizar = MinMaxScaler()
items_not_interacted.loc[:, 'score'] = normalizar.fit_transform(items_not_interacted[['score']]).astype('float64')

/tmp/ipykernel_108763/1171886535.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.         0.00100503 0.00100503 0.00100503 0.00904523 0.13768844
 0.14773869 0.14773869 0.14773869 0.14773869 0.14773869 0.14773869
 0.14773869 0.14773869 0.15577889 0.15577889 0.1638191  0.1718593
 0.38944724 0.38944724 0.39145729 0.40100503 0.40100503 0.40502513
 0.40703518 0.40703518 0.40904523 0.40904523 0.40904523 0.41708543
 0.41708543 0.42110553 0.42914573 0.43316583 0.90954774 0.9678392
 0.9678392  1.         1.         1.        ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  items_not_interacted.loc[:, 'score'] = normalizar.fit_transform(items_not_interacted[['score']]).astype('float64')


In [16]:
# Predecir preferencias para ítems no interactuados
X_recommend = items_not_interacted.drop('oid', axis=1).values
predicted_preferences = model.predict(X_recommend).flatten()

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [17]:
# Ordenar ítems por preferencia predicha y recomendar los mejores
recommendations = items_not_interacted.copy()
recommendations['predicted_preference'] = predicted_preferences
recommendations = recommendations.sort_values(by='predicted_preference', ascending=False).head(10)

In [18]:
print('Recomendaciones para el usuario', user_id)
print(recommendations[['oid', 'predicted_preference']])

Recomendaciones para el usuario 103
                                 oid  predicted_preference
1   31ea1c1b12174428b5a67a6576627de9            230.256012
7   729d37da8f2d46f3af2d891df04949ef            228.744858
8   8f24397e36034cccb71e9d578975c33d            228.744858
6   46850a246d48484b8f104f8aab5679b6            228.744858
14  baf2f8e0167a4e089d2cec16582c9ae9            228.002945
30  e72db1cb2e9f400990cfa3b464d0391a            192.389679
29  b16016fbdbf04317871d334cc4da6eae            192.353958
28  37e4f4a1e8174e9496d21b00d67fc8f1            191.805252
27  17022c9ceac94ec5b2e7bc934c7b2d6f            191.805252
25  c0a2cd8059d44afb822a031066678092            190.315796


> Actualizacion del informe (overleaf) 
- metodologia
- Resultados
- 09-07-2024
- 

In [19]:
df_ejercicios[df_ejercicios['oid'].isin(recommendations['oid'].values)]

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,score
1,31ea1c1b12174428b5a67a6576627de9,0,0,0,1,0,0,0,0,0,0,0,1,257
6,46850a246d48484b8f104f8aab5679b6,0,0,0,1,0,0,0,0,0,0,1,1,259
7,729d37da8f2d46f3af2d891df04949ef,0,0,0,1,0,0,0,0,0,0,1,1,259
8,8f24397e36034cccb71e9d578975c33d,0,0,0,1,0,0,0,0,0,0,1,1,259
14,baf2f8e0167a4e089d2cec16582c9ae9,0,0,0,1,0,0,0,1,0,0,1,1,275
25,c0a2cd8059d44afb822a031066678092,0,0,1,0,0,0,1,0,0,1,1,1,551
27,17022c9ceac94ec5b2e7bc934c7b2d6f,0,0,1,0,0,0,1,1,0,1,1,1,567
28,37e4f4a1e8174e9496d21b00d67fc8f1,0,0,1,0,0,0,1,1,0,1,1,1,567
29,b16016fbdbf04317871d334cc4da6eae,0,0,1,0,0,1,0,0,0,1,1,1,583
30,e72db1cb2e9f400990cfa3b464d0391a,0,0,1,0,0,1,0,1,0,1,1,1,599


In [20]:
df_ejercicios[df_ejercicios['oid'].isin(items_interacted)]

,oid,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1,score
0,2437df93d3f44a87b00834072aeb1ab0,0,0,0,1,0,0,0,0,0,0,0,1,257
2,89f44e7f5842479fb283e43c52ce067b,0,0,0,1,0,0,0,0,0,0,0,1,257
3,718578451f3f4eca87437cadfe98d688,0,0,0,1,0,0,0,0,0,0,1,0,258
4,80c61dae74fa4915bf272ab17dfa62ff,0,0,0,1,0,0,0,0,0,0,1,0,258
5,171b5e86d4fb47268f2692587fbec073,0,0,0,1,0,0,0,0,0,0,1,1,259
9,a3963220090f4e50a266ce53d33b9841,0,0,0,1,0,0,0,0,0,0,1,1,259
10,ac7382763e484d37908da54c076f7577,0,0,0,1,0,0,0,0,0,0,1,1,259
11,d8395f43e4a1454d90346ac5a1ba561a,0,0,0,1,0,0,0,0,0,0,1,1,259
12,7f60644b0a1b484681ae5c8e36166c58,0,0,0,1,0,0,0,1,0,0,0,1,273
13,52620b0c858a4c59bc324b65278d28bd,0,0,0,1,0,0,0,1,0,0,1,1,275


In [ ]:
# from pathlib import Path
# from matplotlib.patches import Patch
# from datetime import datetime, timedelta
# from matplotlib.ticker import MaxNLocator
# from IPython.display import Markdown, display
# from learntools.time_series.utils import plot_periodogram, seasonal_plot
# from learntools.time_series.style import *

# def printmd(string):
#     display(Markdown(string))


# comp_dir = Path('../input/amazon-product-reviews')

# electronics_data = pd.read_csv(
#     comp_dir / "ratings_Electronics (1).csv", 
#     dtype={'rating': 'int8'},
#     names=['userId', 'productId', 'rating', 'timestamp'], 
#     index_col=None, 
#     header=0
# )

# printmd("Number of Rating: {:,}".format(electronics_data.shape[0]))
# printmd("Columns: {}".format(np.array2string(electronics_data.columns.values)))
# printmd("Number of Users: {:,}".format(len(electronics_data.userId.unique())))
# printmd("Number of Products: {:,}".format(len(electronics_data.productId.unique())))
# electronics_data.describe()['rating'].reset_index()

# printmd('**Number of missing values**:')
# pd.DataFrame(electronics_data.isnull().sum().reset_index()).rename(columns={0: "Total missing", "index": "Columns"})

# data_by_date = electronics_data.copy()
# data_by_date.timestamp = pd.to_datetime(electronics_data.timestamp, unit="s")
# data_by_date = data_by_date.sort_values(by="timestamp", ascending=False).reset_index(drop=True)

# printmd("Number of Ratings each day:")
# data_by_date.groupby("timestamp")["rating"].count().tail(10).reset_index()

# data_by_date["year"] = data_by_date.timestamp.dt.year
# data_by_date["month"] = data_by_date.timestamp.dt.month
# rating_by_year = data_by_date.groupby(["year", "month"])["rating"].count().reset_index()
# rating_by_year["date"] = pd.to_datetime(rating_by_year["year"].astype("str") + "-" + rating_by_year["month"].astype("str") + "-1")

# rating_by_year.plot(x="date", y="rating")
# plt.title("Number of Rating over years")
# plt.show()

# rating_by_product = electronics_data.groupby("productId").agg({
#     "userId": "count",
#     "rating": "mean"
# }).rename(columns={"userId": "Number of Ratings", "rating": "Average Rating"}).reset_index()

# cutoff = 50
# top_rated = rating_by_product.loc[rating_by_product["Number of Ratings"] > cutoff].sort_values(by="Average Rating", ascending=False).reset_index(drop=True)

# !/opt/conda/bin/python3.7 -m pip install --upgrade pip
# !pip install -q 



# class RankingModel(tf.keras.Model):
#     def __init__(self):
#         super().__init__()
#         embedding_dimension = 32
#         self.user_embeddings = tf.keras.Sequential([
#             tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=unique_userIds, mask_token=None),
#             tf.keras.layers.Embedding(len(unique_userIds) + 1, embedding_dimension)
#         ])
#         self.product_embeddings = tf.keras.Sequential([
#             tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=unique_productIds, mask_token=None),
#             tf.keras.layers.Embedding(len(unique_productIds) + 1, embedding_dimension)
#         ])
#         self.ratings = tf.keras.Sequential([
#             tf.keras.layers.Dense(256, activation="relu"),
#             tf.keras.layers.Dense(64, activation="relu"),
#             tf.keras.layers.Dense(1)
#         ])
    
#     def call(self, userId, productId):
#         user_embeddings = self.user_embeddings(userId)
#         product_embeddings = self.product_embeddings(productId)
#         return self.ratings(tf.concat([user_embeddings, product_embeddings], axis=1))

# class amazonModel(tfrs.models.Model):
#     def __init__(self):
#         super().__init__()
#         self.ranking_model: tf.keras.Model = RankingModel()
#         self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
#             loss=tf.keras.losses.MeanSquaredError(),
#             metrics=[tf.keras.metrics.RootMeanSquaredError()]
#         )
    
#     def compute_loss(self, features, training=False):
#         rating_predictions = self.ranking_model(features["userId"], features["productId"])
#         return self.task(labels=features["rating"], predictions=rating_predictions)

# cutoff_no_rat = 50  # Only count products which received more than or equal to 50
# cutoff_year = 2011  # Only count Rating after 2011
# recent_data = data_by_date.loc[data_by_date["year"] > cutoff_year]

# print("Number of Rating: {:,}".format(recent_data.shape[0]))
# print("Number of Users: {:,}".format(len(recent_data.userId.unique())))
# print("Number of Products: {:,}".format(len(recent_data.productId.unique())))

# del data_by_date  # Free up memory
# recent_prod = recent_data.loc[recent_data.groupby("productId")["rating"].transform('count').ge(cutoff_no_rat)].reset_index(drop=True).drop(["timestamp", "year", "month"], axis=1)
# del recent_data  # Free up memory

# userIds = recent_prod.userId.unique()
# productIds = recent_prod.productId.unique()
# total_ratings = len(recent_prod.index)

# ratings = tf.data.Dataset.from_tensor_slices({
#     "userId": tf.cast(recent_prod.userId.values, tf.string),
#     "productId": tf.cast(recent_prod.productId.values, tf.string),
#     "rating": tf.cast(recent_prod.rating.values, tf.int8)
# })

# tf.random.set_seed(42)
# shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

# train = shuffled.take(int(total_ratings * 0.8))
# test = shuffled.skip(int(total_ratings * 0.8)).take(int(total_ratings * 0.2))
# unique_productIds = productIds
# unique_userIds = userIds

# model = amazonModel()
# model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
# cached_train = train.shuffle(100_000).batch(8192).cache()
# cached_test = test.batch(4096).cache()

# model.fit(cached_train, epochs=10)
# model.evaluate(cached_test, return_dict=True)

# user_rand = userIds[123]
# test_rating = {}

# for m in test.take(5):
#     test_rating[m["productId"].numpy()] = RankingModel()(tf.convert_to_tensor([user_rand]), tf.convert_to_tensor([m["productId"]]))

# print("Top 5 recommended products for User {}: ".format(user_rand))
# for m in sorted(test_rating, key=test_rating.get, reverse=True):
#     print(m.decode())
